# Reconstitutes data from the event file into the HDF file if the integrated image is defective in some way

In [1]:
import h5py
import numpy as np
from refnx.reduce import catalogue, number_datafile
from refnx.reduce.event import events, process_event_stream
import os.path
import shutil

In [2]:
catalogue(34153, 34175)

,filename,end_time,sample_name,ss1vg,ss2vg,ss3vg,ss4vg,omega,twotheta,bm1_counts,time,daq_dirname,start_time
34153,./PLP0034153.nx.hdf,b'2017-07-02 03:13:39',b'W344 15 mM DSDS D2O Ramp 40C',8.858964,1.8678,0.6522,5.567586,0.8,1.6,6514935,900.001038,DAQ_2017-07-02T02-58-35,1498928315
34154,./PLP0034154.nx.hdf,b'2017-07-02 03:51:11',b'W344 15 mM DSDS D2O Ramp 40C',21.998978,7.7057,2.6898,10.458455,3.3,6.6,30267120,2100.005127,DAQ_2017-07-02T03-16-06,1498929366
34155,./PLP0034155.nx.hdf,b'2017-07-02 04:33:46',b'W344 15 mM DSDS D2O Ramp 35C',8.859086,1.8679,0.6522,5.567586,0.8,1.6,6515741,900.000061,DAQ_2017-07-02T04-18-41,1498933121
34156,./PLP0034156.nx.hdf,b'2017-07-02 05:11:17',b'W344 15 mM DSDS D2O Ramp 35C',21.998856,7.7056,2.6898,10.458333,3.3,6.6,30263685,2100.000488,DAQ_2017-07-02T04-36-12,1498934172
34157,./PLP0034157.nx.hdf,b'2017-07-02 05:43:53',b'W344 15 mM DSDS D2O Ramp 32.5C',8.858964,1.8679,0.6520,5.567708,0.8,1.6,6513512,900.000793,DAQ_2017-07-02T05-28-48,1498937328
34158,./PLP0034158.nx.hdf,b'2017-07-02 06:21:25',b'W344 15 mM DSDS D2O Ramp 32.5C',21.998978,7.7058,2.6896,10.458455,3.3,6.6,30270464,2100.000977,DAQ_2017-07-02T05-46-20,1498938380
34159,./PLP0034159.nx.hdf,b'2017-07-02 06:54:00',b'W344 15 mM DSDS D2O Ramp 30C',8.858964,1.8679,0.6521,5.567586,0.8,1.6,6508481,900.000305,DAQ_2017-07-02T06-38-54,1498941534
34160,./PLP0034160.nx.hdf,b'2017-07-02 07:31:32',b'W344 15 mM DSDS D2O Ramp 30C',21.998978,7.7056,2.6898,10.458333,3.3,6.6,30252648,2100.003418,DAQ_2017-07-02T06-56-27,1498942587
34161,./PLP0034161.nx.hdf,b'2017-07-02 08:04:08',b'W344 15 mM DSDS D2O Ramp 27.5C',8.859208,1.8679,0.6521,5.567586,0.8,1.6,6509861,900.000793,DAQ_2017-07-02T07-49-03,1498945743
34162,./PLP0034162.nx.hdf,b'2017-07-02 08:41:41',b'W344 15 mM DSDS D2O Ramp 27.5C',21.998856,7.7057,2.6898,10.458455,3.3,6.6,30245954,2100.003662,DAQ_2017-07-02T08-06-35,1498946795


In [3]:
def reconsituter(template_number, desired_number, DAQ_folder, title=''):
    # create the desired number file
    template_file_name = number_datafile(template_number)
    new_file_name = number_datafile(desired_number)
    shutil.copyfile(template_file_name, new_file_name)
    
    # get the events
    pth = os.path.join(DAQ_folder, 'DATASET_0', 'EOS.bin')
    e = events(pth, max_frames=10000000)
    
    # get the binning_requirements
    with h5py.File(template_file_name, 'r') as tf:
        ybins = tf['/entry1/data/y_bin'][:]
        xbins = tf['/entry1/data/x_bin'][:]
        tofbins = tf['/entry1/data/time_of_flight'][:]
    
    # rebin the events
    hmm, frame_bins = process_event_stream(e[0], [0, e[0][0][-1] + 1], tofbins, ybins, xbins)
    
    total_counts = int(np.sum(hmm))
    
    with h5py.File(new_file_name, 'r+') as nf:
        nf['/entry1/data/total_counts'][0] = total_counts
        nf['/entry1/instrument/detector/total_counts'][0] = total_counts

        del nf['/entry1/experiment/file_name']
        del nf['/entry1/instrument/detector/daq_dirname']
        data = np.array([bytes(new_file_name, 'UTF-8')])
        nf.create_dataset('/entry1/experiment/file_name', data=data)
        data = np.array([bytes(DAQ_folder, 'UTF-8')])
        nf.create_dataset('/entry1/instrument/detector/daq_dirname', data=data)
    
        del nf['/entry1/data/hmm']
        del nf['/entry1/instrument/detector/hmm']
        nf.create_dataset('/entry1/data/hmm', data=hmm)
        nf.create_dataset('/entry1/instrument/detector/hmm', data=hmm)

        if title:
            del nf['/entry1/sample/name']
            del nf['/entry1/experiment/title']
            data = np.array([bytes(title, 'UTF-8')])
            nf.create_dataset('/entry1/sample/name', data=data)
            nf.create_dataset('/entry1/experiment/title', data=data)

In [4]:
first_configs = {34177:'DAQ_2017-07-02T20-24-15/',
                 34179:'DAQ_2017-07-02T21-59-22/',
                 34181:'DAQ_2017-07-02T23-24-26/',
                 34183:'DAQ_2017-07-03T00-49-30/',
                 34185:'DAQ_2017-07-03T02-14-37/',
                 34187:'DAQ_2017-07-03T03-39-42/'}

second_configs = {34176:'DAQ_2017-07-02T18-34-32/',
                  34178:'DAQ_2017-07-02T20-46-47/',
                  34180:'DAQ_2017-07-02T22-21-53/',
                  34182:'DAQ_2017-07-02T23-46-56/',
                  34184:'DAQ_2017-07-03T01-12-02/',
                  34186:'DAQ_2017-07-03T02-37-07/',
                  34188:'DAQ_2017-07-03T04-02-12/'}

In [5]:
for k, v in first_configs.items():
    print(k, v)
    reconsituter(34153, k, v)

for k, v in second_configs.items():
    print(k, v)
    reconsituter(34154, k, v)

34177 DAQ_2017-07-02T20-24-15/
34179 DAQ_2017-07-02T21-59-22/
34181 DAQ_2017-07-02T23-24-26/
34183 DAQ_2017-07-03T00-49-30/
34185 DAQ_2017-07-03T02-14-37/
34187 DAQ_2017-07-03T03-39-42/
34176 DAQ_2017-07-02T18-34-32/
34178 DAQ_2017-07-02T20-46-47/
34180 DAQ_2017-07-02T22-21-53/
34182 DAQ_2017-07-02T23-46-56/
34184 DAQ_2017-07-03T01-12-02/
34186 DAQ_2017-07-03T02-37-07/
34188 DAQ_2017-07-03T04-02-12/
